## 可逆圧縮
圧縮後のデータから、元のデータを完全に再現できる圧縮アルゴリズムです。  
代表的なものには ランレングス圧縮(連長圧縮) や ハフマン符号 があげられます。

## 非可逆圧縮
圧縮後のデータから、元のデータを完全に復元できない圧縮アルゴリズムです。  
画像や映像、音声など、人間の認知特性上伝わりずらい部分を大幅に減らし、重要となる部分の情報を多く残すようなアルゴリズムになります。  
逆に一部でも異なれば情報として価値がなくなるもの(テキストデータなど)にはあまり利用されません。  

画像では JPEG 形式、音声だと MP3 形式などが代表的なアルゴリズムとしてあげられます。

# ハフマン符号
データの出現頻度に偏りを見出し、圧縮に利用する  
<br>
デビット・ハフマンによって開発された符号で、データの可逆圧縮に利用されます。  
出現頻度の高いデータに短いビット列を、逆に出現頻度の低いデータに長いビット列を割り当てることで、データ全体でのデータ量の削減を実現します。  
<br>
ハフマン符号のアルゴリズムは古典的(代表的)な圧縮アルゴリズムであり、ZIPやJPEGといった圧縮フォーマットでも利用されています。

## 静的ハフマン符号
1回目の走査で出現頻度を調べ、2回目の走査でデータを符号に置き換える O(N^2) の計算量を要する変換アルゴリズムを用いたハフマン符号

## 動的ハフマン符号
出現頻度を調べながら、動的に符号を割り当てていくアルゴリズムを用いたハフマン符号

## 接頭符号
ハフマン符号は**接頭符号**と呼ばれる符号の一種。  
接頭符号とは、任意の符号が他の符号の接頭部にならないような符号のこと。  
```
0
10
111
1100
1101
```
上の符号は、どの符号も他の符号と先頭(接頭部)で一致しないので、接頭符号の条件を満たしている。

<br>

可変長符号が接頭符号であるということは、**一意複合可能**と **瞬時復号可能**という2つの性質を与えてくれます。

### 一意復号可能
ある圧縮されたデータを復号する際に復号結果が一意に定まるということです。  
例えば "0", "1", "01" となる可変長符号で構成されるデータ "01" を復号する場合、"0 1" となるか、"01" となるか一意に決めることができません。  
これは一意複合可能ではない。


### 瞬時復号可能
ある圧縮されたデータがビット列として与えられた場合に、各符号の最後のビットが読み込まれた時点でその符号を確定できるというような性質です。  
ビット列を最後まで読まなくても先頭から順に復号していくことができるという意味で、瞬時復号可能といいます。  
瞬時復号可能ではない符号では、復号処理が複雑になり効率性に影響してしまいます。


# 符号化アルゴリズム
ハフマン符号を得るために、*ハフマン木*と言う二分木を作成する必要がある。  
生成したハフマン木を辿ることで符号化を行える。  

python実装参考サイト：https://k-yuya.hatenablog.com/entry/2018/07/18/231731

In [15]:
from collections import Counter


class Node: #葉を表すクラス
    def __init__(self, value=None, count=None, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.count = count


class Huffman:
    def __init__(self) -> None:
        self.encode_dict = {}
        self.decode_dict = {}

    def encode(self, plane_data):
        unique_data = set(plane_data)
        counter = Counter(plane_data)
        
        # 出現回数を保持したnodeを生成
        nodes = []

        for val in unique_data:
            node_obj = Node(value=val, count=counter[val])
            nodes.append(node_obj)

        # Huffman木の生成
        temp = []
        while len(nodes) >= 2:
            for v in range(2):
                elem = min(nodes, key=lambda x: x.count)
                temp.append(elem)
                nodes.remove(elem)
            new_node = Node(value=None, count=temp[0].count + temp[1].count, left=temp[0], right=temp[1])
            temp = []
            nodes.append(new_node)

        self.trace_tree(nodes[0], "")
        
        result = ""
        for v in plane_data:
            result += self.encode_dict[v]

        return result

    # Huffman木を辿って、それぞれの文字毎にHuffman符号を割り当て.encode_dictに保存する
    def trace_tree(self, node, s):
        if not isinstance(node, Node):
            return

        if node.value:
            print(node.value, node.count, s)
            self.encode_dict[node.value] = s
            self.decode_dict[s] = node.value
        
        self.trace_tree(node.right, s+"0")
        self.trace_tree(node.left, s+"1")


    def decode(self, encoded_data):
        assert(self.decode_dict)
        result = ""
        s = ""
        for bit in encoded_data:
            s += bit
            if s in self.decode_dict:
                result += self.decode_dict[s]
                s = ""
        return result



h = Huffman()
encoded = h.encode("AAABBCDE")
print(encoded, len(encoded))
decoded = h.decode(encoded)
print(decoded)

A 3 00
E 1 010
C 1 011
B 2 10
D 1 11
000000101001111010 18
AAABBCDE
